# Parte 2: Desarrollo de la Red Neuronal

### Forward Propagation

In [406]:
from tkinter import X, Y
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

all_data = pd.read_csv("pasajeros_actualizado.csv")

# Extraer columnas de entrada, escalarlas dividiendo por 255
all_inputs = all_data.iloc[:, 0:16].values
all_outputs = all_data.iloc[:, -1].values
scaler = StandardScaler()
all_inputs = scaler.fit_transform(all_inputs)

# Dividir los conjuntos de datos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs, test_size=1 / 3)
n = X_train.shape[0]
print(n)

# Funciones de activación
relu = lambda x: np.maximum(x, 0)
logistic = lambda x: 1 / (1 + np.exp(-x))

# Construir red neuronal con pesos y sesgos inicializados aleatoriamente
np.random.seed(79)
w_hidden = np.random.rand(12, 16)        
w_hidden2 = np.random.rand(8, 12)        
w_output = np.random.rand(1, 8)         

b_hidden = np.random.rand(12, 1)         
b_hidden2 = np.random.rand(8, 1)        
b_output = np.random.rand(1, 1)         

# Ejecutar entradas a través de la red neuronal para obtener salidas predichas
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden        
    A1 = relu(Z1)
    Z2 = w_hidden2 @ A1 + b_hidden2     
    A2 = relu(Z2)
    Z3 = w_output @ A2 + b_output       
    A3 = logistic(Z3)
    return Z1, A1, Z2, A2, Z3, A3

17317


Precisión No entrenada

In [407]:
# Calculo de precisión
test_predictions = forward_prop(X_test.transpose())[5] 
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test)
accuracy = sum(test_comparisons.astype(int) / X_test.shape[0])

print(accuracy)

0.4381568310428164


### Back Propagation

In [410]:
# Tasa de aprendizaje
L = 0.01

# Derivadas de las funciones de activación
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# Función de backward propagation
def backward_prop(Z1, A1, Z2, A2, Z3, A3, X, Y):
    dC_dA3 = 2 * (A3 - Y)           
    dA3_dZ3 = d_logistic(Z3)
    dZ3_dW3 = A2
    dZ3_dA2 = w_output
    dC_dZ3 = dC_dA3 * dA3_dZ3

    dC_dA2 = dZ3_dA2.T @ dC_dZ3
    dA2_dZ2 = d_relu(Z2)
    dZ2_dW2 = A1
    dZ2_dA1 = w_hidden2
    dC_dZ2 = dC_dA2 * dA2_dZ2

    dC_dA1 = dZ2_dA1.T @ dC_dZ2
    dA1_dZ1 = d_relu(Z1)
    dZ1_dW1 = X
    dC_dZ1 = dC_dA1 * dA1_dZ1

    dC_dW3 = dC_dZ3 @ dZ3_dW3.T
    dC_dB3 = np.sum(dC_dZ3, axis=1, keepdims=True)

    dC_dW2 = dC_dZ2 @ dZ2_dW2.T
    dC_dB2 = np.sum(dC_dZ2, axis=1, keepdims=True)

    dC_dW1 = dC_dZ1 @ dZ1_dW1.T
    dC_dB1 = np.sum(dC_dZ1, axis=1, keepdims=True)
    
    return dC_dW1, dC_dB1, dC_dW2, dC_dB2, dC_dW3, dC_dB3



#### Descenso de Gradiente Estocástico

In [411]:
for i in range(150_000):
    # seleccionar aleatoriamente uno de los datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2, Z3, A3 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación
    # y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2, dW3, dB3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, X_sample, Y_sample)

    # actualizar pesos y sesgos
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_hidden2 -= L * dW2
    b_hidden2 -= L * dB2
    w_output -= L * dW3
    b_output -= L * dB3

Precisión Red Entrenada

In [412]:
# Calculo de precisión de entrenamiento
train_predictions = forward_prop(X_train.transpose())[5]  # Obtener A3 para el conjunto de entrenamiento
train_comparisons = np.equal((train_predictions >= .5).flatten().astype(int), Y_train)
train_accuracy = sum(train_comparisons.astype(int)) / X_train.shape[0]

print(f"Train Accuracy: {train_accuracy}")
    
    
# Calculo de precisión de prueba
test_predictions = forward_prop(X_test.transpose())[5]  # Obtener A3 para el conjunto de prueba
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test)
test_accuracy = sum(test_comparisons.astype(int)) / X_test.shape[0]

print(f"Test Accuracy: {test_accuracy}")


Train Accuracy: 0.9364786048391753
Test Accuracy: 0.9318628017092043


### Scikit-Learn

In [413]:
import pandas as pd
# cargar datos
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

df = pd.read_csv('pasajeros_actualizado.csv')

# Extraer variables de entrada (todas las filas, todas las columnas menos la última)
# Nota que deberíamos hacer algún escalado lineal aquí
X = (df.values[:, :-1])

# Extraer columna de salida (todas las filas, última columna)
Y = df.values[:, -1]

# Separar los datos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3)

nn = MLPClassifier(solver='adam',
                   hidden_layer_sizes=(12, 8, ),
                   activation='relu',
                   max_iter=150_000,
                   learning_rate_init=.01)

nn.fit(X_train, Y_train)

print("Puntaje del conjunto de entrenamiento: %f" % nn.score(X_train, Y_train))
print("Puntaje del conjunto de prueba: %f" % nn.score(X_test, Y_test))


Puntaje del conjunto de entrenamiento: 0.948317
Puntaje del conjunto de prueba: 0.944335
